In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
import pickle
import time
import requests
import json
from joblib import Parallel, delayed


In [2]:
skip_sets = []  # List ones to skip here

root_data = '/data/shared/rework/gold_standard/'

max_predict = 450

# # rsanders@urbn key
# api_key = 'GFIFLWTE6H36GHW2E2R2IYQIET7CKLCXSEENXDQEXDEPL4Q37R3BSAUP2ORC5B4KQDQAWLDXNJW6I45REOWLVXLCR25CABDXV37ZFAA'

# t.szumowski key
api_key = 'R4H75LZTZXUA77D5J3AVHEBHA765GUD7QQAAR52TA23MMW5A4DGABO7ZGURIXDX52ALU55GXBRYAZH5TAA6NVQEVM2PSZSVSGCSHFTY'

url_pred = 'https://api.einstein.ai/v2/vision/predict'

# List of dataset name and model IDs that were trained (from rsanders2@urbn.com account)
# tiny's are from tom's t.szumowski@gmail.com account
dataset_models = {
#     'fashion_mnist_10p': 'SL4CGUDXTQ5ZT5JNXFDBWNFBRI',    
#     'cifar10_20p': 'SRLOAZFACV2KSERFYIAI7GRSI4',    
#     'uo_dress': 'MTC4BV3JMPYCKJ5NY4OYGALZBU',
    'fashion_mnist_tiny': 'KRGDDZ7VNBD42ZXOOUOKMIU6MA',    
    'cifar10_tiny': 'ZHCEZBCHL3IO467A7UZ6KISSUU',    
    'uo_dress_tiny': 'DTSE3ZQVLLZKP5GMBNXFE7EXPY',    
    'mnist_tiny': 'FYT46CRAU7R46BQF7CBXT64NIM',        
    }

In [3]:
def evaluate(dataset_path, model_id):
    
    d = os.path.basename(dataset_path)
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    project_name = '{}_{}'.format(d, timestamp)    
    
    # parse data
    labels = pd.read_csv(os.path.join(dataset_path, 'labels.csv'), header=None, dtype=str)
    train_files = labels[labels[0].str.contains('(train|val)')].values
    test_files = labels[labels[0].str.contains('test')].values    
    
    y_train = train_files[:,1]
    class_labels = np.sort(np.unique(y_train))
    
    # Assign ground truth for test
    y_true = test_files[:,1]

    # Load in files for prediction 
    y_pred = []
    scores = []
    files = {}
    test_filenames = test_files[:, 0]
    test_filenames = [os.path.join(dataset_path, c) for c in test_filenames]
    n_pred = len(test_filenames)
    print('{} prediction files found.'.format(n_pred))
    
    # Clean up sizing
    if n_pred > max_predict:
        print('Number of prediction files ({}) exceeds maximum ({}). sampling by class down to max.'.format(n_pred, max_predict))
        inds=np.random.choice(n_pred, max_predict, replace=False)
        y_true = y_true[inds]
        test_filenames = np.array(test_filenames)[inds].tolist() 
        print("New length: {}".format(len(test_filenames)))
    
    # Worker for predictions
    def predict_worker(filename):
        basename = os.path.basename(filename)
        cmd="curl -s -X POST \
          https://api.einstein.ai/v2/vision/predict \
          -H 'Authorization: Bearer {}' \
          -H 'Cache-Control: no-cache' \
          -H 'Content-Type: multipart/form-data' \
          -H 'content-type: multipart/form-data' \
          -F sampleContent=@{} \
          -F modelId={} \
          -F numResults={}".format(api_key, filename, model_id, len(class_labels))
            
        try:
            out = json.loads(os.popen(cmd).read())
            pred_set = out['probabilities']

            # Get sorted scores from prediction set
            t_pred = None
            t_scores = None            
            t_names = np.array([z['label'] for z in pred_set])
            t_scores = np.array([z['probability'] for z in pred_set])
            sort_inds = np.argsort(t_names)
            t_names = t_names[sort_inds]
            t_scores = t_scores[sort_inds]
            t_pred = t_names[np.argmax(t_scores)]        
        except:
            print('Error occured on prediction: {}. Skipping save.'.format(filename))

        return basename, t_pred, t_scores

    # Run parallel calls to make faster
    t_start = time.time()
    with Parallel(n_jobs=-1, verbose=5) as parallel:
        results = parallel(delayed(predict_worker)(f) for f in test_filenames)
    t_elapsed = time.time() - t_start
    print(len(results))
    print('{:0.3f} secs elapsed for predicting {} images'.format(t_elapsed, len(test_filenames)))

    # Parse the parallel output
    returned_files = [r[0] for r in results]
    y_pred = [r[1] for r in results]
    scores = [r[2] for r in results]
    scores = np.array(scores)

    print('Number of predictions: {}'.format(len(y_pred)))
    print('Number of fails: {}.'.format(sum([yy is None for yy in y_pred])))

    #
    # Save results
    #
    save_file = '{}-results.p'.format(project_name)
    save_dict = {
        'y_true': y_true,
        'y_pred': y_pred,
        'scores': scores,
        'class_labels': class_labels,
        'model_name': project_name,
        'model': None,
        'train_files': train_files,
        'test_files': test_files,
        'returned_files': returned_files
        }
    with open(save_file, 'wb') as f:
        pickle.dump(save_dict, f)
    print('Saved to {}'.format(save_file))

    return save_file

In [4]:
for d,model_id in dataset_models.items():
    if d in skip_sets:
        continue
        
    # Get dataset key and directory
    dataset_path = os.path.join(root_data, d)
    print('EXECUTING DATASET: {}'.format(dataset_path))    
    
    # Run mega routine
    evaluate(dataset_path, model_id)
    
    # Output
    print('Done!')

EXECUTING DATASET: /data/shared/rework/gold_standard/fashion_mnist_tiny
10000 prediction files found.
Number of prediction files (10000) exceeds maximum (450). sampling by class down to max.
New length: 450


/home/szumowskit1/venv/rework/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   35.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


450
35.902 secs elapsed for predicting 450 images
Number of predictions: 450
Number of fails: 0.
Saved to fashion_mnist_tiny_20180913221410-results.p
Done!
EXECUTING DATASET: /data/shared/rework/gold_standard/uo_dress_tiny
545 prediction files found.
Number of prediction files (545) exceeds maximum (450). sampling by class down to max.
New length: 450


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   32.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


450
32.413 secs elapsed for predicting 450 images
Number of predictions: 450
Number of fails: 0.
Saved to uo_dress_tiny_20180913221446-results.p
Done!
EXECUTING DATASET: /data/shared/rework/gold_standard/mnist_tiny
10000 prediction files found.
Number of prediction files (10000) exceeds maximum (450). sampling by class down to max.
New length: 450


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   35.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


450
35.672 secs elapsed for predicting 450 images
Number of predictions: 450
Number of fails: 0.
Saved to mnist_tiny_20180913221519-results.p
Done!
EXECUTING DATASET: /data/shared/rework/gold_standard/cifar10_tiny
10000 prediction files found.
Number of prediction files (10000) exceeds maximum (450). sampling by class down to max.
New length: 450


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   30.6s


450
31.056 secs elapsed for predicting 450 images
Number of predictions: 450
Number of fails: 0.
Saved to cifar10_tiny_20180913221555-results.p
Done!


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:   31.1s finished
